In [2]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset, ConcatDataset
from torchvision import transforms, datasets
from PIL import Image
import os
import zipfile
import torch.optim as optim
import torch.nn as nn
import numpy as np
from sklearn.model_selection import KFold
import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
class MixedDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        if isinstance(self.dataset, TensorDataset):
            x, y = self.dataset[index]
        else:
            x, y = self.dataset[index]
        
        # Ensure x is a tensor
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x)
        
        # Ensure y is a tensor
        if not isinstance(y, torch.Tensor):
            y = torch.tensor(y)
        
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.dataset)

def add_noise(image, noise_factor=0.5):
    noise = torch.randn_like(image) * noise_factor
    noisy_image = image + noise
    return torch.clamp(noisy_image, 0., 1.)

# Load experimental data
def load_all_experimental_data(test_digits_folder):
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []
    participant_data = {}

    transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    for filename in os.listdir(test_digits_folder):
        if filename.endswith('.zip') and filename.startswith('experiment_results_participant'):
            participant_number = int(filename.split('participant')[1].split('.')[0])
            zip_filepath = os.path.join(test_digits_folder, filename)

            participant_train_images = []
            participant_train_labels = []
            participant_test_images = []
            participant_test_labels = []

            with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                for img_filename in zip_ref.namelist():
                    if img_filename.endswith('.png'):
                        with zip_ref.open(img_filename) as file:
                            img = Image.open(file).convert('L')
                            img_tensor = transform(img)
                            
                            digit = int(img_filename.split('_')[0])
                            
                            if 'composite' in img_filename:
                                test_images.append(img_tensor)
                                test_labels.append(digit)
                                participant_test_images.append(img_tensor)
                                participant_test_labels.append(digit)
                            else:
                                train_images.append(img_tensor)
                                train_labels.append(digit)
                                participant_train_images.append(img_tensor)
                                participant_train_labels.append(digit)

            participant_data[participant_number] = {
                'train': (torch.stack(participant_train_images), torch.tensor(participant_train_labels)),
                'test': (torch.stack(participant_test_images), torch.tensor(participant_test_labels))
            }

    return (torch.stack(train_images), torch.tensor(train_labels), 
            torch.stack(test_images), torch.tensor(test_labels),
            participant_data)

# Load and augment MNIST data
def load_augmented_mnist():
    transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
    ])
    mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    noisy_transform = transforms.Compose([
        transforms.Lambda(lambda x: add_noise(x, noise_factor=0.5)),
    ])
    invert_transform = transforms.Compose([
        transforms.Lambda(lambda x: 1 - x),
    ])

    noisy_mnist_train = MixedDataset(mnist_train, noisy_transform)
    noisy_mnist_test = MixedDataset(mnist_test, noisy_transform)
    inverted_mnist_train = MixedDataset(mnist_train, invert_transform)
    inverted_mnist_test = MixedDataset(mnist_test, invert_transform)

    combined_train = ConcatDataset([mnist_train, noisy_mnist_train, inverted_mnist_train])
    combined_test = ConcatDataset([mnist_test, noisy_mnist_test, inverted_mnist_test])

    return combined_train, combined_test

# Model Definition
class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(16 * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience):
    model = model.to(device)
    best_val_acc = 0
    epochs_no_improve = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}')
        
        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = model(val_images)
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_acc = 100 * correct_val / total_val
        print(f'Validation Accuracy: {val_acc:.2f}%')
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
    
    return model

# K-Fold Cross Validation
def k_fold_cross_validation(dataset, num_folds=5, num_epochs=50, patience=5):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_results = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for fold, (train_index, val_index) in enumerate(kf.split(range(len(dataset)))):
        print(f"Fold {fold + 1}/{num_folds}")
        train_subset = torch.utils.data.Subset(dataset, train_index)
        val_subset = torch.utils.data.Subset(dataset, val_index)

        train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

        model = LeNet5_16x16(num_classes=10).to(device)
        
        # Calculate class weights for weighted cross-entropy
        labels = torch.tensor([dataset[i][1] for i in train_index])
        class_counts = torch.bincount(labels)
        class_weights = 1. / class_counts.float()
        class_weights = class_weights / class_weights.sum()
        criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
        
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience)

        # Evaluate on validation set
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = model(val_images)
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_accuracy = 100 * correct_val / total_val
        fold_results.append(val_accuracy)
        print(f'Fold {fold + 1} Validation Accuracy: {val_accuracy:.2f}%')

    return fold_results

In [5]:
if __name__ == "__main__":
    # Load experimental data
    exp_train_images, exp_train_labels, exp_test_images, exp_test_labels, participant_data = load_all_experimental_data('test_digits')
    exp_dataset = TensorDataset(exp_train_images, exp_train_labels)

    # Load augmented MNIST data
    mnist_train, mnist_test = load_augmented_mnist()

    # Combine experimental data with augmented MNIST data
    combined_dataset = ConcatDataset([MixedDataset(exp_dataset), MixedDataset(mnist_train)])

    # Perform K-Fold Cross Validation
    fold_results = k_fold_cross_validation(combined_dataset, num_folds=5, num_epochs=50, patience=5)
    print(f"Average Validation Accuracy across folds: {np.mean(fold_results):.2f}%")

    # Train final model on all data
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    final_train_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)
    final_val_loader = DataLoader(MixedDataset(mnist_test), batch_size=32, shuffle=False)

    final_model = LeNet5_16x16(num_classes=10).to(device)
    
    # Calculate class weights for the final model
    all_labels = []
    for dataset in combined_dataset.datasets:
        if isinstance(dataset, TensorDataset):
            all_labels.extend(dataset.tensors[1].tolist())
        else:
            all_labels.extend([label for _, label in dataset])
    
    labels = torch.tensor(all_labels)
    class_counts = torch.bincount(labels)
    class_weights = 1. / class_counts.float()
    class_weights = class_weights / class_weights.sum()
    
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(final_model.parameters(), lr=0.001)

    final_model = train_model(final_model, final_train_loader, final_val_loader, criterion, optimizer, num_epochs=50, patience=5)

    # Save the final model
    torch.save(final_model.state_dict(), "lenet5_trained_model_heavy.pth")
    print("Final model saved as lenet5_trained_model_heavy.pth")

Fold 1/5
Epoch [1/50], Loss: 6526.6644
Validation Accuracy: 70.36%
Epoch [2/50], Loss: 4811.8453
Validation Accuracy: 72.54%
Epoch [3/50], Loss: 4482.2102
Validation Accuracy: 73.67%
Epoch [4/50], Loss: 4261.4427
Validation Accuracy: 74.01%
Epoch [5/50], Loss: 4113.2235
Validation Accuracy: 74.63%
Epoch [6/50], Loss: 4006.0122
Validation Accuracy: 74.87%
Epoch [7/50], Loss: 3943.1851
Validation Accuracy: 74.93%
Epoch [8/50], Loss: 3901.9558
Validation Accuracy: 75.33%
Epoch [9/50], Loss: 3871.6060
Validation Accuracy: 75.55%
Epoch [10/50], Loss: 3835.1228
Validation Accuracy: 75.61%
Epoch [11/50], Loss: 3797.7419
Validation Accuracy: 74.68%
Epoch [12/50], Loss: 3785.8795
Validation Accuracy: 75.99%
Epoch [13/50], Loss: 3774.8289
Validation Accuracy: 75.85%
Epoch [14/50], Loss: 3764.0283
Validation Accuracy: 75.76%
Epoch [15/50], Loss: 3745.5406
Validation Accuracy: 75.77%
Epoch [16/50], Loss: 3736.7553
Validation Accuracy: 75.78%
Epoch [17/50], Loss: 3717.4506
Validation Accuracy: 75.1

In [ ]:
if __name__ == "__main__":
    # Load experimental data
    exp_train_images, exp_train_labels, exp_test_images, exp_test_labels, participant_data = load_all_experimental_data('test_digits')
    exp_dataset = TensorDataset(exp_train_images, exp_train_labels)

    # Load augmented MNIST data
    mnist_train, mnist_test = load_augmented_mnist()

    # Combine experimental data with augmented MNIST data
    combined_dataset = ConcatDataset([MixedDataset(exp_dataset), MixedDataset(mnist_train)])

    # Perform K-Fold Cross Validation
    fold_results = k_fold_cross_validation(combined_dataset, num_folds=5, num_epochs=50, patience=5)
    print(f"Average Validation Accuracy across folds: {np.mean(fold_results):.2f}%")

    # Train final model on all data
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    final_train_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)
    final_val_loader = DataLoader(MixedDataset(mnist_test), batch_size=32, shuffle=False)

    final_model = LeNet5_16x16(num_classes=10).to(device)
    
    # Calculate class weights for the final model
    all_labels = []
    for dataset in combined_dataset.datasets:
        if isinstance(dataset, TensorDataset):
            all_labels.extend(dataset.tensors[1].tolist())
        else:
            all_labels.extend([label for _, label in dataset])
    
    labels = torch.tensor(all_labels)
    class_counts = torch.bincount(labels)
    class_weights = 1. / class_counts.float()
    class_weights = class_weights / class_weights.sum()
    
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(final_model.parameters(), lr=0.001)

    final_model = train_model(final_model, final_train_loader, final_val_loader, criterion, optimizer, num_epochs=50, patience=5)

    # Save the final model
    torch.save(final_model.state_dict(), "lenet5_trained_model_heavy.pth")
    print("Final model saved as lenet5_trained_model_heavy.pth")

Fold 1/5
Epoch [1/50], Loss: 6526.6644
Validation Accuracy: 70.36%
Epoch [2/50], Loss: 4811.8453
Validation Accuracy: 72.54%
Epoch [3/50], Loss: 4482.2102
Validation Accuracy: 73.67%
Epoch [4/50], Loss: 4261.4427
Validation Accuracy: 74.01%
Epoch [5/50], Loss: 4113.2235
Validation Accuracy: 74.63%
Epoch [6/50], Loss: 4006.0122
Validation Accuracy: 74.87%
Epoch [7/50], Loss: 3943.1851
Validation Accuracy: 74.93%
Epoch [8/50], Loss: 3901.9558
Validation Accuracy: 75.33%
Epoch [9/50], Loss: 3871.6060
Validation Accuracy: 75.55%
Epoch [10/50], Loss: 3835.1228
Validation Accuracy: 75.61%
Epoch [11/50], Loss: 3797.7419
Validation Accuracy: 74.68%
Epoch [12/50], Loss: 3785.8795
Validation Accuracy: 75.99%
Epoch [13/50], Loss: 3774.8289
Validation Accuracy: 75.85%
Epoch [14/50], Loss: 3764.0283
Validation Accuracy: 75.76%
Epoch [15/50], Loss: 3745.5406
Validation Accuracy: 75.77%
Epoch [16/50], Loss: 3736.7553
Validation Accuracy: 75.78%
Epoch [17/50], Loss: 3717.4506
Validation Accuracy: 75.1